In [3]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install ultralytics --upgrade

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install onnxruntime onnxruntime-tools

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 161.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 KB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 KB 123.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 34.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 53.7 MB/s eta 0:00:00


In [4]:
import tarfile
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import shutil
import multiprocessing

# === Use container disk (/tmp) for faster IO ===
split_dir = Path('.')  # You moved .tar.gz files here
base_dir = split_dir / 'splits_aligned_fused' / 'splits_aligned_fused_2025-06-06_01-11-05'
base_dir.mkdir(parents=True, exist_ok=True)

# === List your .tar.gz archives ===
split_archives = [
    'splits_aligned_fused_2025-06-06_01-11-05_images_trains.tar.gz',
    'splits_aligned_fused_2025-06-06_01-11-05_lables_trains.tar.gz',
    'splits_aligned_fused_2025-06-06_01-11-05_val_test.tar.gz'
]
tar_files = [split_dir / fname for fname in split_archives]

# === Extraction function ===
def extract_tar(tar_path):
    print(f"🔓 Starting: {tar_path.name}")
    with tarfile.open(tar_path, "r:gz") as tar:
        members = tar.getmembers()
        total = len(members)
        print(f"📁 {total} files in {tar_path.name}")
        for i, member in enumerate(members, 1):
            tar.extract(member, path=base_dir, filter='data')
            if i % 10000 == 0:
                print(f"📈 {tar_path.name}: {i:,}/{total:,} ({(i/total)*100:.1f}%)")
    print(f"✅ Done: {tar_path.name}")
    return tar_path.name

# === Parallel Extraction ===
print(f"🔍 Extracting {len(tar_files)} archives...")
workers = min(len(tar_files), multiprocessing.cpu_count())
with ThreadPoolExecutor(max_workers=workers) as executor:
    futures = [executor.submit(extract_tar, f) for f in tar_files]
    for i, f in enumerate(futures, 1):
        f.result()
        print(f"📦 Completed {i}/{len(futures)}")

# === Create YOLO folder structure ===
image_train = base_dir / 'images' / 'train'
label_train = base_dir / 'labels' / 'train'
image_val = base_dir / 'images' / 'val'
label_val = base_dir / 'labels' / 'val'
image_test = base_dir / 'images' / 'test'
label_test = base_dir / 'labels' / 'test'

for path in [image_train, label_train, image_val, label_val, image_test, label_test]:
    path.mkdir(parents=True, exist_ok=True)

# === Move Files ===
def move_all(src_glob, dst_folder, description):
    count = 0
    for file in base_dir.glob(src_glob):
        shutil.move(str(file), dst_folder / file.name)
        count += 1
    print(f"📦 Moved {count} {description} to {dst_folder}")

move_all("**/images/train/*.png", image_train, "train images")
move_all("**/labels/train/*.txt", label_train, "train labels")
move_all("**/images/val/*.png", image_val, "val images")
move_all("**/labels/val/*.txt", label_val, "val labels")
move_all("**/images/test/*.png", image_test, "test images")
move_all("**/labels/test/*.txt", label_test, "test labels")

print(f"\n✅ Done. Folder ready at: {base_dir}")

🔍 Extracting 3 archives...
🔓 Starting: splits_aligned_fused_2025-06-06_01-11-05_images_trains.tar.gz
🔓 Starting: splits_aligned_fused_2025-06-06_01-11-05_lables_trains.tar.gz
🔓 Starting: splits_aligned_fused_2025-06-06_01-11-05_val_test.tar.gz
📁 213207 files in splits_aligned_fused_2025-06-06_01-11-05_val_test.tar.gz
📁 605899 files in splits_aligned_fused_2025-06-06_01-11-05_lables_trains.tar.gz
📈 splits_aligned_fused_2025-06-06_01-11-05_val_test.tar.gz: 10,000/213,207 (4.7%)
📈 splits_aligned_fused_2025-06-06_01-11-05_lables_trains.tar.gz: 10,000/605,899 (1.7%)
📈 splits_aligned_fused_2025-06-06_01-11-05_val_test.tar.gz: 20,000/213,207 (9.4%)
📈 splits_aligned_fused_2025-06-06_01-11-05_lables_trains.tar.gz: 20,000/605,899 (3.3%)
📈 splits_aligned_fused_2025-06-06_01-11-05_val_test.tar.gz: 30,000/213,207 (14.1%)
📈 splits_aligned_fused_2025-06-06_01-11-05_lables_trains.tar.gz: 30,000/605,899 (5.0%)
📈 splits_aligned_fused_2025-06-06_01-11-05_val_test.tar.gz: 40,000/213,207 (18.8%)
📈 splits_a

In [4]:
from pathlib import Path
dataset_root = Path('./splits_aligned_fused/splits_aligned_fused_2025-06-06_01-11-05')

splits = ['train', 'val', 'test']
for split in splits:
    img_dir = dataset_root / 'images' / split
    lbl_dir = dataset_root / 'labels' / split

    img_files = list(img_dir.glob('*.png'))
    lbl_files = list(lbl_dir.glob('*.txt'))

    print(f"\n📂 {split.upper()}")
    print(f"🖼️  Images: {len(img_files)}")
    print(f"🏷️  Labels: {len(lbl_files)}")

    img_stems = {f.stem for f in img_files}
    lbl_stems = {f.stem for f in lbl_files}
    missing_labels = img_stems - lbl_stems
    missing_images = lbl_stems - img_stems

    if missing_labels:
        print(f"⚠️ Missing labels for: {list(missing_labels)[:5]}")
    if missing_images:
        print(f"⚠️ Missing images for: {list(missing_images)[:5]}")
    if not missing_labels and not missing_images:
        print("✅ All image-label pairs match.")


📂 TRAIN
🖼️  Images: 605896
🏷️  Labels: 605896
✅ All image-label pairs match.

📂 VAL
🖼️  Images: 70509
🏷️  Labels: 70509
✅ All image-label pairs match.

📂 TEST
🖼️  Images: 36091
🏷️  Labels: 36091
✅ All image-label pairs match.


In [6]:
import yaml

class_names = ["chair", "box", "desk", "door_frame"]

data_yaml = {
    'path': str(dataset_root),
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': len(class_names),
    'names': class_names
}

yaml_file_path = dataset_root / 'data.yaml'
with open(yaml_file_path, 'w') as f:
    yaml.dump(data_yaml, f)

print(f"📄 data.yaml saved at: {yaml_file_path}")

📄 data.yaml saved at: splits_aligned_fused/splits_aligned_fused_2025-06-06_01-11-05/data.yaml


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(
    data=str(yaml_file_path),
    epochs=120,
    batch=1024,         # Safe value for 40 GB VRAM
    imgsz=384,
    workers=10,          # Balanced for 30+ CPU cores
    rect=True,          # All images are same size (64x384)
    cache='ram',        # Leverage 200 GB RAM for faster training
    seed=42,
    save=True,
    save_period=10,
    patience=50,

    # ✅ Safe augmentations
    flipud=0.2,
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,

    # ❌ Disable risky augmentations
    mosaic=0.0,         # Disable mosaic
    mixup=0.0,          # Disable mixup
    copy_paste=0.0,     # Disable copy-paste
    cutmix=0.0,         # Disable cutmix
    auto_augment=None,  # Disable auto augment policies
    augment=False,      # Disable general heavy augmentation
    erasing=0.0,        # Disable random erasing

    verbose=True
)

print("✅ Training complete!")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/soheil/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:00<00:00, 490MB/s]


Ultralytics 8.3.158 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:0 (NVIDIA H100 80GB HBM3, 81008MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=None, batch=1024, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=splits_aligned_fused/splits_aligned_fused_2025-06-06_01-11-05/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=120, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=384, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=50, per

100%|████████████████████████████████████████████████████████████████████████| 755k/755k [00:00<00:00, 146MB/s]

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

     48/120      9.72G     0.7328     0.4009     0.9314       2586        384: 100%|██████████| 592/592 [02:32<
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/3


                   all      70509     269259      0.951      0.949      0.984      0.695

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     49/120      9.72G     0.7307     0.4002     0.9309       2599        384: 100%|██████████| 592/592 [02:33<
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/3


                   all      70509     269259      0.951      0.948      0.984      0.694

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     50/120      9.72G     0.7298     0.3965     0.9316       3737        384:  50%|█████     | 296/592 [01:16<

In [8]:
import yaml
from pathlib import Path
from ultralytics import YOLO
model = YOLO('runs/detect/train/weights/last.pt')

results = model.train(
    data=str(yaml_file_path),
    epochs=120,
    batch=1024,         # Safe value for 40 GB VRAM
    imgsz=384,
    workers=10,          # Balanced for 30+ CPU cores
    rect=True,          # All images are same size (64x384)
    cache='ram',        # Leverage 200 GB RAM for faster training
    seed=42,
    save=True,
    save_period=10,
    patience=50,
    resume=True,

    # ✅ Safe augmentations
    flipud=0.2,
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,

    # ❌ Disable risky augmentations
    mosaic=0.0,         # Disable mosaic
    mixup=0.0,          # Disable mixup
    copy_paste=0.0,     # Disable copy-paste
    cutmix=0.0,         # Disable cutmix
    auto_augment=None,  # Disable auto augment policies
    augment=False,      # Disable general heavy augmentation
    erasing=0.0,        # Disable random erasing

    verbose=True
)

print("✅ Training complete!")

Ultralytics 8.3.158 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:0 (NVIDIA H100 80GB HBM3, 81008MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=None, batch=1024, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=splits_aligned_fused/splits_aligned_fused_2025-06-06_01-11-05/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=120, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=384, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/train/weights/last.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask

[W622 17:02:45.130406885 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 17:02:45.133337069 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 17:02:45.135197684 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 17:02:45.138130682 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 17:02:45.139593804 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 17:02:45.139815198 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 17:02:45.140383957 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 17:02:45.140736174 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 17:02:45.141251473 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 17:02:45.141868399 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 17:0

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 122.5±59.6 MB/s, size: 1.2 KB)


train: Scanning /home/soheil/splits_aligned_fused/splits_aligned_fused_2025-06-06_01-11-05/labels/train.cache... 605896 images, 0 


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

                   all      70509     269259      0.939      0.877      0.947      0.658

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    120/120      9.73G     0.4788     0.2603     0.8549       2616        384: 100%|██████████| 592/592 [02:35<00:00,  3.80it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [02:12<00:00,  3.


                   all      70509     269259      0.933       0.87      0.942       0.65

71 epochs completed in 5.681 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.158 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:0 (NVIDIA H100 80GB HBM3, 81008MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [02:45<00:00,  4.


                   all      70509     269259      0.953      0.937      0.981      0.708
                 chair      69391      69391      0.944      0.921      0.973       0.64
                   box      69924      69924      0.967      0.997      0.993      0.717
                  desk      70509      70509      0.936      0.896      0.973      0.764
            door_frame      59435      59435      0.965      0.934      0.984      0.712
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train
✅ Training complete!


In [10]:
from ultralytics import YOLO
import torch
import json
import shutil
import os
from onnxruntime.quantization import quantize_dynamic, QuantType

# --- Paths ---
weights_dir = 'runs/detect/train/weights'
best_model_path = f'{weights_dir}/best.pt'
data_yaml_path = './splits_aligned_fused/splits_aligned_fused_2025-06-06_01-11-05/data.yaml'
webots_dir = './webots_model_assets_simpel_fused_fulltrain_120'
os.makedirs(webots_dir, exist_ok=True)

# --- Load Trained YOLOv8 Model ---
model = YOLO(best_model_path)

# --- Export TorchScript (FP32) ---
print("🔁 Exporting TorchScript...")
model.export(format='torchscript', imgsz=[64, 384])
torchscript_path = f'{weights_dir}/best.torchscript'

# --- Dynamic Quantization for TorchScript ---
print("⚙️ Quantizing TorchScript model...")
model_fp32 = torch.jit.load(torchscript_path)
model_int8 = torch.quantization.quantize_dynamic(model_fp32, {torch.nn.Linear}, dtype=torch.qint8)
torchscript_quant_path = f'{weights_dir}/best_quant_dynamic.torchscript'
torch.jit.save(model_int8, torchscript_quant_path)

# --- Export ONNX (FP32) ---
print("🔁 Exporting ONNX...")
model.export(format='onnx', imgsz=[64, 384], dynamic=True)
onnx_path = f'{weights_dir}/best.onnx'

# --- Dynamic Quantization for ONNX ---
print("⚙️ Quantizing ONNX model...")
onnx_quant_path = f'{weights_dir}/best_quant_dynamic.onnx'
quantize_dynamic(onnx_path, onnx_quant_path, weight_type=QuantType.QInt8)

# # --- Export TensorRT (FP16) ---
# print("🚀 Exporting TensorRT (FP16)...")
# model.export(format='engine', imgsz=[64, 384])
# tensorrt_path = f'{weights_dir}/best.engine'

# --- Evaluate model ---
print("🔍 Evaluating on test set...")
metrics = model.val(data=data_yaml_path, split='test')

# --- Save metrics ---
metrics_path = 'test_metrics.json'
with open(metrics_path, 'w') as f:
    json.dump(metrics.results_dict, f, indent=2)
print("✅ Metrics saved.")

# --- Copy all files to Webots directory ---
shutil.copy(best_model_path, f'{webots_dir}/yolov8n_lidar.pt')
shutil.copy(torchscript_path, f'{webots_dir}/yolov8n_lidar.torchscript')
shutil.copy(torchscript_quant_path, f'{webots_dir}/yolov8n_lidar_quant_dynamic.torchscript')
shutil.copy(onnx_path, f'{webots_dir}/yolov8n_lidar.onnx')
shutil.copy(onnx_quant_path, f'{webots_dir}/yolov8n_lidar_quant_dynamic.onnx')
# shutil.copy(tensorrt_path, f'{webots_dir}/yolov8n_lidar_fp16.engine')
shutil.copy(metrics_path, f'{webots_dir}/test_metrics.json')

print(f"📦 All models and metrics saved to: {webots_dir}")

🔁 Exporting TorchScript...
Ultralytics 8.3.158 🚀 Python-3.10.12 torch-2.7.1+cu126 CPU (Intel Xeon Platinum 8468)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 64, 384) BCHW and output shape(s) (1, 8, 504) (5.9 MB)

TorchScript: starting export with torch 2.7.1+cu126...


[W622 22:54:14.029845866 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:14.030530354 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:14.030705910 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:14.030890033 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:14.031048361 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:14.031214574 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:14.031370732 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:14.031515890 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:14.031665878 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:14.031796171 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:5

TorchScript: export success ✅ 0.8s, saved as 'runs/detect/train/weights/best.torchscript' (11.8 MB)

Export complete (0.9s)
Results saved to /home/soheil/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.torchscript imgsz=64,384  
Validate:        yolo val task=detect model=runs/detect/train/weights/best.torchscript imgsz=64,384 data=splits_aligned_fused/splits_aligned_fused_2025-06-06_01-11-05/data.yaml  WARNING ⚠️ non-PyTorch val requires square images, 'imgsz=[64, 384]' will not work. Use export 'imgsz=384' if val is required.
Visualize:       https://netron.app


[W622 22:54:15.785820322 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:15.785929356 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:15.786027529 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:15.786116892 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:15.786609829 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:15.786674903 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:15.786755663 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:15.786839847 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:15.786913340 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:54:15.787215499 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W622 22:5

⚙️ Quantizing TorchScript model...
🔁 Exporting ONNX...
Ultralytics 8.3.158 🚀 Python-3.10.12 torch-2.7.1+cu126 CPU (Intel Xeon Platinum 8468)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 64, 384) BCHW and output shape(s) (1, 8, 504) (5.9 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.56', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
WARNING ⚠️ requirements: ❌ [Errno 13] Permission denied: 'uv'

ONNX: starting export with onnx 1.18.0 opset 19...


...

⚙️ Quantizing ONNX model...
🔍 Evaluating on test set...
Ultralytics 8.3.158 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:0 (NVIDIA H100 80GB HBM3, 81008MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 8.1±1.9 MB/s, size: 1.2 KB)


val: Scanning /home/soheil/splits_aligned_fused/splits_aligned_fused_2025-06-06_01-11-05/labels/test... 36091 images, 0 background


val: New cache created: /home/soheil/splits_aligned_fused/splits_aligned_fused_2025-06-06_01-11-05/labels/test.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2256/2256 [01:28<00:00,


                   all      36091     137555       0.95      0.932      0.978      0.704
                 chair      35552      35552       0.94      0.889      0.959      0.636
                   box      35931      35931       0.97      0.996      0.993      0.704
                  desk      36007      36007      0.934      0.902      0.974      0.768
            door_frame      30065      30065      0.956      0.941      0.987      0.708
Speed: 0.0ms preprocess, 0.2ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/val
✅ Metrics saved.
📦 All models and metrics saved to: ./webots_model_assets_simpel_fused_fulltrain_120


In [11]:
import shutil

# Zip the entire runs directory
shutil.make_archive('./runs_yolo_120', 'zip', './runs')
shutil.make_archive('./webots_model_assets_simpel_fused_fulltrain_120', 'zip', './webots_model_assets_simpel_fused_fulltrain_120')

'/home/soheil/webots_model_assets_simpel_fused_fulltrain_120.zip'